In [3]:
%%sh
cat > zadanie3.c << EOF
#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>
#include <time.h>
#include <sys/time.h>
#define N 9


MPI_Status status;
double A[N][N];
double B[N], C[N];

int main(int argc, char **argv) {
int processCount, processId, slaveTaskCount, source, dest, rows, offset;

struct timeval start, stop;

MPI_Init(&argc, &argv);
MPI_Comm_rank(MPI_COMM_WORLD, &processId);
MPI_Comm_size(MPI_COMM_WORLD, &processCount);


slaveTaskCount = processCount - 1;

if (processId == 0) {

double start = MPI_Wtime();


srand ( time(NULL) );
for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
    A[i][j]= rand()%10;
  }
  B[i]= rand()%10;
}

printf("\n Mnozenie macierzy przez wektor za pomoca MPI \n");
printf("\nMacierz A\n\n");
for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
    printf("%.0f\t", A[i][j]);
  }
  printf("\n");
}
printf("\nWektor B\n\n");
for (int i = 0; i<N; i++) {
  printf("%.0f\t", B[i]);
  printf("\n");
}


rows = N/slaveTaskCount;


offset = 0;


for (dest=1; dest <= slaveTaskCount; dest++)
{
  MPI_Send(&offset, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
  MPI_Send(&rows, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
  MPI_Send(&A[offset][0], rows*N, MPI_DOUBLE,dest,1, MPI_COMM_WORLD);
  MPI_Send(&B, N, MPI_DOUBLE, dest, 1, MPI_COMM_WORLD);
  offset = offset + rows;
}


for (int i = 1; i <= slaveTaskCount; i++)
{
  source = i;
  MPI_Recv(&offset, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
  MPI_Recv(&rows, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
  MPI_Recv(&C[offset], rows*N, MPI_DOUBLE, source, 2, MPI_COMM_WORLD, &status);
}


printf("\nWynikowy wektor C:\n\n");
for (int i = 0; i<N; i++) {
  printf("%.0f\t", C[i]);

}
printf ("\n");

double end = MPI_Wtime();
printf ("\n");
printf("Czas obliczen: %f",end - start);
}

// Kod do wykonania przez procesy Slave
if (processId > 0) {

  source = 0;
  MPI_Recv(&offset, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&rows, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&A, rows*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&B, N*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
  for (int i = 0; i<rows; i++) {
    for (int j = 0; j<N; j++)
      C[i] = C[i] + A[i][j] * B[j];
  }

  MPI_Send(&offset, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);

  MPI_Send(&rows, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);

  MPI_Send(&C, rows*N, MPI_DOUBLE, 0, 2, MPI_COMM_WORLD);
}
MPI_Finalize();
}
EOF
mpicc zadanie3.c && mpirun -n 10 --allow-run-as-root a.out


 Mnozenie macierzy przez wektor za pomoca MPI 

Macierz A

7	6	0	6	1	0	7	2	7	
6	1	9	9	2	7	6	4	0	
0	0	6	8	8	7	2	6	1	
8	8	9	0	6	3	1	3	7	
3	5	1	4	4	4	2	2	0	
1	2	5	8	0	3	7	4	9	
7	0	8	9	0	4	2	1	9	
2	4	6	5	8	2	1	2	7	
7	8	6	4	8	6	9	7	1	

Wektor B

0	
7	
3	
0	
4	
8	
1	
3	
0	

Wynikowy wektor C:

59	116	126	141	94	72	61	101	184	

Czas obliczen: 0.001024